In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from time import time
from collections import Counter

In [ ]:
suicide_detection_df = pd.read_csv('/content/drive/MyDrive/Tumblr Vietnam/Tusat2.csv', header=0)
suicide_detection_df.reset_index(drop=True, inplace=True)
suicide_detection_df.replace({"class": {"suicide": 1, "non-suicide": 0}}, inplace=True)
suicide_detection_df.drop(columns=['text'], inplace=True)
suicide_detection_df = suicide_detection_df.rename(columns={"cleaned_text": "text"})
suicide_detection_df

,class,text
0,1,sex wife threaten suicide recently leave wife ...
1,0,weird not affect compliment come know girl fee...
2,0,finally hear bad year swear fucking god annoying
3,1,need help help cry hard
4,1,lose hello adam struggle year afraid past year...
...,...,...
232069,0,not like rock not go
232070,0,tell friend not lonely deprive buy little nigh...
232071,0,pee probably taste like salty tea drink pee co...
232072,1,usual stuff find not post sympathy pity know f...


In [ ]:
suicide_detection_df = suicide_detection_df.dropna(subset=['text'])



In [ ]:
SEED = 4222
EPOCHS = 5
train_text, test_text, train_labels, test_labels = train_test_split(suicide_detection_df['text'], suicide_detection_df['class'],
                                                                    random_state=SEED,
                                                                    test_size=0.2,
                                                                    stratify=suicide_detection_df['class'])


In [ ]:
vocab = Counter()
tokens_list = [(s.split()) for s in train_text]
for i in tokens_list:
  vocab.update(i)
min_occurance = 2
tokens = [k for k,c in vocab.items() if c >= min_occurance]
print(len(tokens))


27879


In [ ]:
def save_list(lines, filename)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_list(vocab, '/content/drive/MyDrive/Tumblr Vietnam/vocab.txt')

In [ ]:
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text
vocab_filename = '/content/drive/MyDrive/Tumblr Vietnam/vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

In [ ]:
def clean_line(line, vocab):
  tokens = line.split()
  tokens_clean = [w for w in tokens if w in vocab]
  return [tokens_clean]

def process_lines(data, vocab):
  lines = list()
  for i in data:
    line = clean_line(i, vocab)
    lines += line
  return lines

In [ ]:
train_clean = process_lines(train_text, vocab)
test_clean = process_lines(test_text, vocab)

In [ ]:
model = Word2Vec(vector_size=300, window=10, min_count=1, epochs=EPOCHS, seed=SEED)

t = time()
model.build_vocab(train_clean, progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()
model.train(train_clean, total_examples=model.corpus_count, epochs=EPOCHS, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.04 mins
Time to train the model: 2.03 mins


In [ ]:
filename = '/content/drive/MyDrive/Tumblr Vietnam/embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)